In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib as mpl
import os
import numpy as np
import imageio
from datetime import datetime
from tqdm import tqdm

# Load Project Data

In [2]:
projects = [
    'bayc'
]

In [3]:
df_summaries = {}

column_names = [
    "date", 
    "days_since_mint", 
    "from_address", 
    "to_address", 
    "token_id", 
    "blk_number", 
    "eth_value", 
    "usd_value", 
    "from_value", 
    "to_value", 
    "from_value_usd", 
    "to_value_usd"
]

for project in projects:
    np_data = np.load(f"./memory/{project}/full.npy", allow_pickle=True)
    df_summaries[project] = pd.DataFrame(data=np_data, columns=column_names).infer_objects()

# Draw a Network

In [4]:
def draw_network(graph, filename, date, dataset, pos):
    
    # if we already have some nodes, we want to keep the positions of these nodes.
    # this does make the drawing not too busy.
    if pos:
        pos = nx.spring_layout(graph, pos=pos)
    else:
        pos = nx.spring_layout(graph)

    # get the weights for drawing the width of the edges
    widths = nx.get_edge_attributes(graph, 'weight')

    # build a heat map for the number of trades of the accounts.
    # we need to treat the 0x0 address a bit differently.
    nx.set_node_attributes(graph, {'0x0000000000000000000000000000000000000000':-1}, 'trades')
    color_lookup = { v:k['trades'] for v, k in graph.nodes.data()}
    low, *_, high = sorted(color_lookup.values())
    norm = mpl.colors.Normalize(vmin=low, vmax=high, clip=True)
    mapper = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.viridis)
    # map everything according to the defined scale except the 0x0 address node, which is just white.
    null_mapper = ['white' if i == -1 else mapper.to_rgba(i) for i in color_lookup.values()]

    # map the value of a node to it's size. We have a fixed size for the 0x0 node.
    nx.set_node_attributes(graph, {'0x0000000000000000000000000000000000000000':10}, 'eth_value')
    size_lookup = {v:k['eth_value'] for v, k in graph.nodes.data()}

    fig, ax = plt.subplots(figsize=(100, 100))
    ax.set_facecolor("#000000")
    
    # Visualize graph components
    nx.draw_networkx_edges(graph, pos, alpha=0.3, edge_color="m", width=list(widths.values()))
    nx.draw_networkx_nodes(graph, pos, node_color=null_mapper, alpha=0.8, node_size=[v * 10 for v in size_lookup.values()])
    
    # add an info box at the top
    textstr = "Project: %s\nDate: %s" % (dataset, date)

    # place a text box in upper left in axes coords
    props = dict(boxstyle='round', facecolor='black', alpha=1.0)
    ax.text(0.75, 0.95, textstr, transform=ax.transAxes, fontsize=70, verticalalignment='top', bbox=props, color='white')

    # save frame
    plt.savefig(filename)
    plt.close()

    # plt.show()
    return pos

In [5]:
# this is the same code that we use in the ingest notebook.
# except that we draw a plot every day and we use the eth value for the edge weight.
def build_graph(df, project):    
    # Building a network per block
    # we will use a weighted and directed graph.
    graph = nx.MultiDiGraph()

    startDate = df['date'].iloc[0]

    filenames = [] # store the generated filenames.
    pos = None # empty positions for the graph.
    # loop over the pandas dataframe.
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        # read the values from the dataframe.
        # token_id  blk_timestamp eth_value 
        date = row['date']
        from_address = row['from_address']
        to_address = row['to_address']
        token_id = row['token_id']
        blk_number = row['blk_number']
        eth_value = row['eth_value']
        usd_value = row['usd_value']
        from_value = row['from_value']
        to_value = row['to_value']
        from_value_usd = row['from_value_usd']
        to_value_usd = row['to_value_usd']
        
        # make sure both addresses are in the graph.
        if from_address not in graph:
            graph.add_node(from_address)
        if to_address not in graph:
            graph.add_node(to_address)

        # set the attributes on this node.
        nx.set_node_attributes(graph, {from_address: from_value, to_address: to_value}, 'eth_value')
        nx.set_node_attributes(graph, {from_address: from_value_usd, to_address: to_value_usd}, 'usd_value')

        # keep track of how many trades a wallet has done.
        trades = nx.get_node_attributes(graph, "trades")
        if from_address in trades:
            nx.set_node_attributes(graph, {from_address:trades[from_address] + 1}, 'trades')
        else:
            nx.set_node_attributes(graph, {from_address:1}, 'trades')
        if to_address in trades:
            nx.set_node_attributes(graph, {to_address:trades[to_address] + 1}, 'trades')
        else:
            nx.set_node_attributes(graph, {to_address:1}, 'trades')

        # check if this NFT has already been sold and if yes, remove the old sale.
        # this might be a candidate for memoization - c.b.
        remove_edges = []
        for (u,v,d) in graph.edges.data():
            if d['token_id'] == token_id:
                remove_edges.append((u,v))
        # we need to remove them in a seperate step, since otherwise we change the datastructure that we are iterating over.
        for (u,v) in remove_edges:
            graph.remove_edge(u,v)

        # add an edge for the transaction. # Note changed to usd_value
        graph.add_edge(from_address, to_address, weight=eth_value, token_id=token_id) # keep track of token id by adding it to the edge.

        # plot the network every day.
        if (date - startDate).total_seconds() > 86400:
            # create file name and append it to a list
            filename = f'./data/tmp/%s.png' % index
            filenames.append(filename)
            pos = draw_network(graph, filename, date.strftime('%Y-%m-%d'), project, pos)
            startDate = date
    
    # build gif
    with imageio.get_writer('./data/gifs/%s.gif' % project, mode='I') as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)
            
    # remove files
    for filename in set(filenames):
        os.remove(filename)

In [6]:
for project in projects:
    np_data = np.load(f"./memory/{project}/full.npy", allow_pickle=True)
    df = pd.DataFrame(data=np_data, columns=column_names).infer_objects()    
    
    build_graph(df, "Bored Ape Yacht Club")

  0%|          | 0/50950 [00:00<?, ?it/s]/opt/homebrew/Caskroom/miniforge/base/envs/.venv/lib/python3.9/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)
 21%|██        | 10606/50950 [00:24<01:11, 567.04it/s]/opt/homebrew/Caskroom/miniforge/base/envs/.venv/lib/python3.9/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)
100%|██████████| 50950/50950 [7:32:22<0